In [1]:
#pip install prophet
#pip install pathos

In [57]:
import sys
sys.path.append(str('/home/jupyter/SearchForecasting/'))
from configparser import ConfigParser

from repo.MyConfigParser import MyConfigParser

from repo.LoadData import DataLoader
from repo.BuildModel import GenerateModelFile, CreateModel
from repo.Accuracy import AccuracyCalculator, ProphetParameterSearch, PlotAccuracyCharts
from repo.FileGenerator import *

In [58]:
updated_config = ConfigParser()
config_in_path = "/home/jupyter/SearchForecasting/UK/June 2024 Update/config ups uk.ini"
updated_config.read(config_in_path)

getdict = MyConfigParser()
config_dict=getdict.GetDict(updated_config)

region = config_dict['CLIENTINFO']['country']
cutoff_month = config_dict['PROPHETSETTING']['cut_off_point']
base_start = config_dict['PROPHETSETTING']['reference_start_date1']
base_end = config_dict['PROPHETSETTING']['reference_end_date1']
periods = config_dict['PROPHETSETTING']['periods']
base_periods  = config_dict['PROPHETSETTING']['base_periods']

training_query = config_dict["PROPHETSETTING"]["training_query"]
_input = config_dict["PROPHETSETTING"]["input"]
_output = config_dict["PROPHETSETTING"]["output"]

output_bucket = config_dict["FILEPATH"]["output_bucket"]
folder_name = config_dict["FILEPATH"]["folder_name"]


today = datetime.date.today().strftime("%m.%d.%Y") #'03.10.2023' #
print(today)
base_path = config_dict["FILEPATH"]["base_path"]

boundary_df_in_path = base_path + config_dict["FILEPATH"]["boundary_df_in_path"]
daily_df_out_path = base_path + config_dict["FILEPATH"]["daily_df_out_path"]
analytical_file_path = base_path + config_dict["FILEPATH"]["analytical_file_path"]
cuts_df_in_path = base_path + config_dict["FILEPATH"]["cuts_df_in_path"]
plot_name = base_path + config_dict["FILEPATH"]["plot_name"]
constraint_file_name = base_path + config_dict["FILEPATH"]["constraint_file_name"]

05.20.2024


In [4]:
# today = datetime.date.today().strftime("%m.%d.%Y") #'03.10.2023' #
# print(today)
# base_path = '/home/jupyter/CA/24 Feb Update/'

# config_in_path = base_path + "config ups ca reg.ini"

# boundary_df_in_path = base_path + 'Floor Cap Setting 2023 Global.csv'
# daily_df_out_path = base_path + f'Daily Data {today}.csv'
# analytical_file_path = base_path + 'Analytical_File_NY'+ '.csv'
# cuts_df_in_path = base_path + region + ' Breakdown (1).csv'
# # constraint_file_out_path = base_path + 'Constraint File Modified Nov.csv'
# plot_name = base_path + "Accuracy_By_Month.jpg"
# constraint_file_name = "Constraint File.csv"

In [59]:
daily_df = GetBQData(training_query, _input, _output)

df = daily_df.copy()
df['market'] = '-'
df = df.groupby(['region', 'market', 'funnel', 'ds']).sum().reset_index()

# log Transformation
# df['cost'] = np.log(df['cost'])
# df.loc[(df['cost'] < 0) | (np.isinf(df['cost'])), 'cost'] = 0

df.ds = pd.to_datetime(df.ds)
df = df[~df[['ds', 'y']].isin([np.nan, np.inf, -np.inf]).any(axis=1)]
df = df[df.ds < cutoff_month]
df['funnel'] = df['funnel'].str.replace('Competitor|Generic', 'Non_Brand', regex=True)

# remove inactive days
df.drop(df[((df['ds'] >= '2021-08-01')&(df['ds'] <= '2021-10-31')) & ((df.account_type == 'Bing Ads') & (df.funnel == 'Brand'))].index,
       axis = 0, inplace = True)
df.drop(df[((df['ds'] >= '2019-03-01')&(df['ds'] <= '2019-08-31')) & ((df.account_type == 'Bing Ads') & (df.funnel == 'Brand'))].index,
       axis = 0, inplace = True)

df.drop(df[((df['ds'] >= '2021-07-01')&(df['ds'] <= '2021-10-31')) & ((df.account_type == 'Bing Ads') & (df.funnel == 'Generic'))].index,
       axis = 0, inplace = True)

# add regressors
df['anomaly_2021'] = df['ds'].apply(lambda x: 1 if (x >= datetime.datetime(2021,8,1))\
                                     & (x <= datetime.datetime(2021,10,31)) else 0)
df['covid_year_dummy'] = 0
df.loc[(df['ds'] >= '2020-01-01') & (df['ds'] <= '2020-12-31'), 'covid_year_dummy'] = 1

df['weekday_or_weekend'] = df['ds'].apply(lambda x: 1 if x.dayofweek > 4 else 0)

# set cap and floor for log regression
df['Month of Ds'] = df['ds'].dt.month_name()

df.tail()

/home/jupyter/SearchForecasting/repo/QueryingData.py:34: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  search_df.fillna(method = 'ffill', inplace = True)


,region,market,funnel,ds,account_type,X,y,anomaly_2021,covid_year_dummy,weekday_or_weekend,Month of Ds
810,Europe,-,-,2024-04-20,DesktopMobile & Tablet,243.15,75.0,0,0,1,April
811,Europe,-,-,2024-04-21,DesktopMobile & Tablet,243.15,126.0,0,0,1,April
812,Europe,-,-,2024-04-22,DesktopMobile & Tablet,243.15,1170.0,0,0,0,April
813,Europe,-,-,2024-04-23,DesktopMobile & Tablet,243.15,1074.0,0,0,0,April
814,Europe,-,-,2024-04-24,DesktopMobile & Tablet,243.15,1148.0,0,0,0,April


In [62]:
df['ds'].sort_values().unique()

<DatetimeArray>
['2022-01-01 00:00:00', '2022-01-02 00:00:00', '2022-01-03 00:00:00',
 '2022-01-04 00:00:00', '2022-01-05 00:00:00', '2022-01-06 00:00:00',
 '2022-01-07 00:00:00', '2022-01-08 00:00:00', '2022-01-09 00:00:00',
 '2022-01-10 00:00:00',
 ...
 '2024-04-15 00:00:00', '2024-04-16 00:00:00', '2024-04-17 00:00:00',
 '2024-04-18 00:00:00', '2024-04-19 00:00:00', '2024-04-20 00:00:00',
 '2024-04-21 00:00:00', '2024-04-22 00:00:00', '2024-04-23 00:00:00',
 '2024-04-24 00:00:00']
Length: 815, dtype: datetime64[ns]

# Model constraint file 
    Upload model constraint file, which has for each split (account_type & funnel) the AOV and ROAS

In [53]:
boundary_df = pd.read_csv(boundary_df_in_path)
boundary_df.rename(columns = {'Market': 'market','Account Type': 'account_type', 'Funnel':'funnel'}, inplace = True)
boundary_df

# filter the data in df to just the one that is going to be used (only the ones with splits from the boundary_df) <br>
# this is set with an inner merge of boundary_df and df. 

df = pd.merge(df, boundary_df, on = ['market', 'funnel'])\
                                    .rename(columns={'account_type_x':'account_type'})\
                                    .drop('account_type_y', axis=1)
df.to_csv('df.csv', index=False)


# add additional variables needed but that are not regressors in the prophet model 

# set floor to the model forecast -> the model should not predict values below this value 
df['floor'] = 0

# set cap to the mnodel forecast -> max values, where the model should not forecast above this values 
df['cap'] = df.ROAS * df.X / df.AOV
df.loc[df.cap == 0, 'cap'] = df.cap.mean() # if cap is 0 set it to the mean cap 

df

# Add Covid Data

covid_df = GetCovidData('United Kingdom')
# covid_df.loc[covid_df['market']=='Canada','market'] = 'Canada'
final_df = df.merge(covid_df, how = 'left', on = ['ds', 'market'])
final_df = final_df.fillna(0)

final_df.to_csv(daily_df_out_path)
final_df.to_csv('dfcovidconfirmed.csv', index=False)


In [6]:
# updated_df = Combine(df, 'L')
# updated_df.head()

,ds,region,market,account_type,funnel,Identifier,X,y,anomaly_2021,covid_year_dummy,weekday_or_weekend,Month of Ds,new_funnel,generic_cost,competitor_cost,generic_cost_split,competitor_cost_split,Weekend
0,2017-08-10,Europe,UK,-,Brand,L,2409.129976,580.0,0.0,0.0,0.0,AugustAugust,NaN,0.000000,0.0,0.5,0.5,0
1,2017-08-10,Europe,UK,-,Non_Brand,L,1220.189985,5.0,0.0,0.0,0.0,AugustAugust,NaN,1220.189985,0.0,1.0,0.0,0
2,2017-08-11,Europe,UK,-,Brand,L,2212.890007,1041.0,0.0,0.0,0.0,AugustAugust,NaN,0.000000,0.0,0.5,0.5,0
3,2017-08-11,Europe,UK,-,Non_Brand,L,1126.269987,10.0,0.0,0.0,0.0,AugustAugust,NaN,1126.269987,0.0,1.0,0.0,0
4,2017-08-12,Europe,UK,-,Brand,L,0.000000,22.0,0.0,0.0,2.0,AugustAugust,NaN,0.000000,0.0,0.5,0.5,1


In [7]:
# covid_df = GetCovidData('United Kingdom')
# covid_df.tail()

,ds,market,new_confirmed
986,2022-09-13,United Kingdom,12828
987,2022-09-14,United Kingdom,8491
988,2022-09-15,United Kingdom,0
989,2022-09-16,United Kingdom,0
990,2022-09-17,United Kingdom,0


In [8]:
# final_df = updated_df.merge(covid_df, how = 'left', on = ['ds', 'market'])
# final_df = final_df.fillna(0)


# final_df['floor'] = 0

# boundary_df = pd.read_csv(boundary_df_in_path)
# boundary_df.rename(columns = {'Market': 'market', 'Account Type': 'account_type', 'Funnel':'funnel'}, inplace = True)

# final_df = pd.merge(final_df, boundary_df, on = ['market', 'account_type', 'funnel'])
# final_df['cap'] = final_df.ROAS * final_df.X / final_df.AOV
# final_df.loc[final_df.cap == 0, 'cap'] = final_df.cap.max()
# final_df = final_df[final_df.ds < cutoff_month].copy()
# final_df.to_csv(daily_df_out_path)
# final_df

/var/tmp/ipykernel_140193/3292308880.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df = final_df.fillna(0)


,ds,region,market,account_type,funnel,Identifier,X,y,anomaly_2021,covid_year_dummy,...,generic_cost,competitor_cost,generic_cost_split,competitor_cost_split,Weekend,new_confirmed,floor,AOV,ROAS,cap
0,2017-08-10,Europe,UK,-,Brand,L,2409.129976,580.0,0.0,0.0,...,0.000000,0.00,0.500000,0.500000,0,0,0,23.16,6.01,625.167148
1,2017-08-10,Europe,UK,-,Non_Brand,L,1220.189985,5.0,0.0,0.0,...,1220.189985,0.00,1.000000,0.000000,0,0,0,24.36,5.37,268.982768
2,2017-08-11,Europe,UK,-,Brand,L,2212.890007,1041.0,0.0,0.0,...,0.000000,0.00,0.500000,0.500000,0,0,0,23.16,6.01,574.243046
3,2017-08-11,Europe,UK,-,Non_Brand,L,1126.269987,10.0,0.0,0.0,...,1126.269987,0.00,1.000000,0.000000,0,0,0,24.36,5.37,248.278729
4,2017-08-12,Europe,UK,-,Brand,L,0.000000,22.0,0.0,0.0,...,0.000000,0.00,0.500000,0.500000,1,0,0,23.16,6.01,4563.432444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4895,2024-04-22,Europe,UK,-,Non_Brand,L,163.430000,13.0,0.0,0.0,...,19.790000,143.64,0.121092,0.878908,0,0,0,24.36,5.37,36.027057
4896,2024-04-23,Europe,UK,-,Brand,L,5927.946900,1057.0,0.0,0.0,...,0.000000,0.00,0.500000,0.500000,0,0,0,23.16,6.01,1538.297101
4897,2024-04-23,Europe,UK,-,Non_Brand,L,50.350000,17.0,0.0,0.0,...,34.310000,16.04,0.681430,0.318570,0,0,0,24.36,5.37,11.099323
4898,2024-04-24,Europe,UK,-,Brand,L,5234.656600,1121.0,0.0,0.0,...,0.000000,0.00,0.500000,0.500000,0,0,0,23.16,6.01,1358.388867


# Analytical File Generator

In [54]:
generator = AnalyticalFileGenerator(config_dict, final_df)
analytical_file_df = generator.generate()
analytical_file_df.columns


splits_names = list(analytical_file_df[['market','account_type','funnel']].drop_duplicates().itertuples(index=False, name=None)) # get the actual splits in df 
splits_names = ['_'.join(item).replace(' ', '_') for item in splits_names]# organize the element splits names joining by '_' 
analytical_file_df.drop(columns=['Month of Ds'], inplace=True)
analytical_file_columns = ['region', 'market', 'account_type', 'funnel', 'ds', 'X', 'Month','Year'] + splits_names + ['weekday_or_weekend','AOV','ROAS','floor','cap'] # add other columns needed in analytical file

# drop columns that arent needed in analytical df, splits that aren't optimized 
for col in analytical_file_df.columns: # Loop over each column in the DataFrame
    if not any(col.startswith(prefix) for prefix in analytical_file_columns): # Check if the column starts with any of the prefixes in combined_strings
        print(f'column {col} dropped from analytical_file_df')
        analytical_file_df.drop(columns=col, inplace=True) # if the columns does not starts with any element in analytical_file_columns drop it 

analytical_file_df

/home/jupyter/SearchForecasting/repo/QueryingData.py:34: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  search_df.fillna(method = 'ffill', inplace = True)


KeyError: "['Month of Ds'] not found in axis"

In [9]:
# analytical_input = GetBQData(config_dict["PROPHETSETTING"]["analytical_file_query"], config_dict["PROPHETSETTING"]["input"], config_dict["PROPHETSETTING"]["output"])
# analytical_input = Combine(analytical_input, 'L')

# generator = AnalyticalFileGenerator(config_dict, analytical_input)
# analytical_file_df = generator.generate()
# analytical_file_df = pd.merge(analytical_file_df, boundary_df, on = ['market', 'account_type', 'funnel'])
# analytical_file_df.drop(analytical_file_df.filter(regex = 'Apr|May').columns, axis=1, inplace = True)
# # generator.SaveFile(analytical_file_name, folder_name) # X is input value which should be the sum of all variable splits
# analytical_file_df.to_csv(analytical_file_path)
# analytical_file_df

/home/jupyter/SearchForecasting/repo/QueryingData.py:34: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  search_df.fillna(method = 'ffill', inplace = True)
/home/jupyter/SearchForecasting/repo/FileGenerator.py:149: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  analytical_file_NY_prep = analytical_file_NY_prep.fillna(0)


,ds,region,market,account_type,funnel,Identifier,X,new_funnel,generic_cost,competitor_cost,...,UK_-_Brand_Sep_2024,UK_-_Non_Brand_Aug_2024,UK_-_Non_Brand_Dec_2024,UK_-_Non_Brand_Jul_2024,UK_-_Non_Brand_Jun_2024,UK_-_Non_Brand_Nov_2024,UK_-_Non_Brand_Oct_2024,UK_-_Non_Brand_Sep_2024,AOV,ROAS
0,2024-04-25,Europe,UK,-,Brand,L,5067.790000,0,0.0000,0.00,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,23.16,6.01
1,2024-04-25,Europe,UK,-,Non_Brand,L,142.457400,0,8.6474,133.81,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,24.36,5.37
2,2024-04-26,Europe,UK,-,Brand,L,12104.050000,0,0.0000,0.00,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,23.16,6.01
3,2024-04-26,Europe,UK,-,Non_Brand,L,195.269600,0,42.9896,152.28,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,24.36,5.37
4,2024-04-27,Europe,UK,-,Brand,L,1401.110000,0,0.0000,0.00,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,23.16,6.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,2024-12-29,Europe,UK,-,Non_Brand,L,696.461616,0,0.0000,0.00,...,0.0,0.0,696.461616,0.0,0.0,0.0,0.0,0.0,24.36,5.37
498,2024-12-30,Europe,UK,-,Brand,L,674.420000,0,0.0000,0.00,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,23.16,6.01
499,2024-12-30,Europe,UK,-,Non_Brand,L,60.898500,0,60.8985,0.00,...,0.0,0.0,60.898500,0.0,0.0,0.0,0.0,0.0,24.36,5.37
500,2024-12-31,Europe,UK,-,Brand,L,4183.490000,0,0.0000,0.00,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,23.16,6.01


# cuts df

import cuts_df. Initial prophet hyper parameters the ones set on previous refresh

In [30]:
cuts_df = pd.read_csv(cuts_df_in_path).query("market == 'UK'")
cuts_df['periods'] = periods
cuts_df['periods_for_acc_cal'] = 183
cuts_df['cutoff_month'] = cutoff_month

query_item_list = [query_item for query_item in [config_dict["CLIENTINFO"]["breakdown1"], \
                             # config_dict["CLIENTINFO"]["breakdown2"], \
                             config_dict["CLIENTINFO"]["breakdown3"], \
                             config_dict["CLIENTINFO"]["breakdown4"]]
                             if query_item != None]

cuts_df

,market,account_type,funnel,changepoint_prior_scale,seasonality_prior_scale,holidays_prior_scale,seasonality_mode,input_mode,weekday_or_weekend,include_holiday,...,generic_cost_split,competitor_cost_split,new_confirmed,Nov_Dec_2018,Jul_Dec_2019,power,growth,periods,periods_for_acc_cal,cutoff_month
9,UK,-,Brand,0.5,0.01,10.00,additive,additive,multiplicative,na,...,na,na,na,na,na,na,logistic,251.0,183,2024-04-25
10,UK,-,Non_Brand,0.1,10.00,0.01,additive,multiplicative,na,na,...,na,na,na,na,na,na,linear,251.0,183,2024-04-25


# Parallel Param Search

Probably to drop

In [25]:
# cuts_df1 = cuts_df.drop(['Nov_Dec_2018', 'Jul_Dec_2019'], axis=1)

In [22]:
# cuts_df[cuts_df.funnel == "Non_Brand"]

Index(['market', 'account_type', 'funnel', 'changepoint_prior_scale',
       'seasonality_prior_scale', 'holidays_prior_scale', 'seasonality_mode',
       'input_mode', 'weekday_or_weekend', 'include_holiday',
       'covid_year_exclude', 'covid_year_dummy', 'anomaly_2021',
       'generic_cost_split', 'competitor_cost_split', 'new_confirmed',
       'Nov_Dec_2018', 'Jul_Dec_2019', 'power', 'growth', 'periods',
       'periods_for_acc_cal', 'cutoff_month'],
      dtype='object')

In [19]:
# final_df.columns

Index(['ds', 'region', 'market', 'account_type', 'funnel', 'Identifier', 'X',
       'y', 'anomaly_2021', 'covid_year_dummy', 'weekday_or_weekend',
       'Month of Ds', 'new_funnel', 'generic_cost', 'competitor_cost',
       'generic_cost_split', 'competitor_cost_split', 'Weekend',
       'new_confirmed', 'floor', 'AOV', 'ROAS', 'cap'],
      dtype='object')

In [26]:
# # %load_ext autoreload
# # %autoreload 2
# search_engine = ProphetParameterSearch(power_flag = False, base_path=base_path) # , logistic_flag = False
# # search_engine.add_power_params()
# grid_brand = search_engine.search(cuts_df1[cuts_df1.funnel == "Non_Brand"], final_df)
# # grid

  0%|          | 0/1 [00:00<?, ?it/s]19:24:15 - cmdstanpy - INFO - Chain [1] start processing
19:24:15 - cmdstanpy - INFO - Chain [1] start processing
19:24:15 - cmdstanpy - INFO - Chain [1] done processing
19:24:15 - cmdstanpy - INFO - Chain [1] done processing
19:24:16 - cmdstanpy - INFO - Chain [1] start processing


It's overpredicted.
Monthly Error: 83.29 Period MAE: 19.6


19:24:16 - cmdstanpy - INFO - Chain [1] start processing
19:24:16 - cmdstanpy - INFO - Chain [1] done processing
19:24:16 - cmdstanpy - INFO - Chain [1] done processing


It's overpredicted.
Monthly Error: 83.29 Period MAE: 19.6


19:24:18 - cmdstanpy - INFO - Chain [1] start processing
19:24:18 - cmdstanpy - INFO - Chain [1] start processing
19:24:18 - cmdstanpy - INFO - Chain [1] done processing
19:24:18 - cmdstanpy - INFO - Chain [1] done processing


...

 Period MAE: 25.729999999999997


20:09:50 - cmdstanpy - INFO - Chain [1] start processing
20:09:50 - cmdstanpy - INFO - Chain [1] done processing


It's underpredicted.
Monthly Error: 100.0 Period MAE: 100.0
It's overpredicted.
Monthly Error: 107.37 Period MAE: 25.729999999999997


20:09:51 - cmdstanpy - INFO - Chain [1] start processing
20:09:51 - cmdstanpy - INFO - Chain [1] done processing
20:09:53 - cmdstanpy - INFO - Chain [1] done processing


It's underpredicted.
Monthly Error: 100.0 Period MAE: 100.0
It's overpredicted.
Monthly Error: 107.37 Period MAE: 25.729999999999997
It's underpredicted.
Monthly Error: 100.0 Period MAE: 100.0


In [79]:
# grid_brand

NameError: name 'grid_brand' is not defined

# Accuracy based on previous refresh

##### First check the accuracy based on previous refresh

In [48]:
calculator = AccuracyCalculator() 
accuracy_df = calculator.CalculateAccuracy(cuts_df, final_df, query_item_list) # calculate accuracy based on cuts_df previous hyper parameters
#calculator.SaveFile()
calculator.SaveFile(output_bucket, folder_name, 'Accuracy.csv')

13:43:32 - cmdstanpy - INFO - Chain [1] start processing
13:43:32 - cmdstanpy - INFO - Chain [1] done processing


It's overpredicted.
Monthly Error: 109.35 Period MAE: 105.36000000000001


/home/jupyter/SearchForecasting/repo/Accuracy.py:267: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.monthly_acc_df_temp = pd.concat([self.monthly_acc_df_temp, monthly_acc_df], ignore_index=True)
13:43:43 - cmdstanpy - INFO - Chain [1] start processing
13:43:45 - cmdstanpy - INFO - Chain [1] done processing


It's overpredicted.
Monthly Error: 144.15 Period MAE: 79.64
Data Frame file saved


In [49]:
accuracy = accuracy_df.groupby([ 'funnel']).mean(numeric_only=1) # group by split 
accuracy['Accuracy'] = np.where(accuracy['MAPE'] > 100, 'MAPE above 100%', 100 - accuracy['MAPE']) # create accuracy from MAPE 
accuracy['Forecast Period Accuracy'] = np.where(accuracy['Forecast Period MAE'] > 100, 'Forecast Period MAE above 100%', 100 - accuracy['Forecast Period MAE']) # create period accuracy from period MAPE
accuracy.drop(columns=['MAPE','Forecast Period MAE'], inplace=True) # drop error columns 
accuracy.rename(columns = {'Percentage Error': 'Mean Percentage Error'}, inplace = True) # rename columns
accuracy


,Actual,Forecast,Mean Percentage Error,Accuracy,Forecast Period Accuracy
funnel,,,,,
Brand,17546.000000,36033.181429,-109.347143,MAPE above 100%,Forecast Period MAE above 100%
Non_Brand,1172.142857,2105.675714,-143.080000,MAPE above 100%,20.36


In [50]:
PlotAccuracyCharts(accuracy_df, cuts_df, query_item_list)


ValueError: not enough values to unpack (expected 3, got 2)

# Prophet parameter search

In [ ]:
param_grid = {
    'changepoint_prior_scale': [ 0.1, 0.05, 0.001],   
    'seasonality_prior_scale': [10.0, 0.1, 0.01, 1.0 ], 
    'holidays_prior_scale': [10.0, 1.0,  0.1, 0.01  ], 
    'seasonality_mode': ['additive','multiplicative'], 
    'input_mode': ['additive','multiplicative'], 
    'weekday_or_weekend':['na', 'additive', 'multiplicative'],  
    'include_holiday': ['na', 'Default', 'custom'],
    'covid_year_exclude':['na'],
    'covid_year_dummy':['na'],#, 'additive', 'multiplicative'],
    'anomaly_2021':['na'],#, 'additive', 'multiplicative'],
    'generic_cost_split':['na'],#, 'additive', 'multiplicative'
    'competitor_cost_split':['na'], #, 'additive', 'multiplicative'
    'new_confirmed': ['na'],#, 'additive', 'multiplicative'
            }


In [ ]:
# set the prophet parameter search with the hyper parameter 
#grid created above and
#power flag as false (not needed)
search_engine = ProphetParameterSearch(param_grid = param_grid, power_flag = False, base_path=base_path) 
grid_brand = search_engine.search(cuts_df, final_df) # performs the grid search over 
#the hyper parameter grid. exports a best result .csv table with best hyper parameters by split and a record df .csv table with full hyper parameters results


# Set new Cuts_df with the new hyper parameters for prophet after the ProphetParameterSearch

In [ ]:
# set the new cuts df 
New_Cuts_df = pd.read_csv(base_path + 'record_df 2024-05-17 11:35:18 best result.csv', index_col = 0) # read the best results df from prophet hyper parameter search
New_Cuts_df = New_Cuts_df.drop_duplicates() # drop duplicate rows, best result sometimes creates duplicate rows 
#New_Cuts_df = New_Cuts_df.sort_values(by=['market','account_type', 'funnel', 'MAPE'], ascending=[True, True, True, True]) # sort by
New_Cuts_df = New_Cuts_df.groupby(['market','account_type', 'funnel']).apply(lambda x: x[x['MAPE'] == x['MAPE'].min()], include_groups=False) # group by breakdowns and select only the minimum MAPE values
New_Cuts_df.reset_index(level=['market', 'account_type', 'funnel'], inplace=True)
New_Cuts_df['na_count'] = New_Cuts_df.map(lambda x: x == 'na').sum(axis=1) # count number of 'na' hyper parameters 
New_Cuts_df = New_Cuts_df.groupby(['market','account_type', 'funnel']).apply(lambda x: x[x['na_count'] == x['na_count'].max()], include_groups=False) # select the set of hyper parameters that has more 'na' for parsimony porpuses
New_Cuts_df.reset_index(level=['market', 'account_type', 'funnel'], inplace=True)
New_Cuts_df.drop(columns=['na_count','MAPE', 'period_AE'], inplace=True) # drop columns that are not needed in cuts_df
New_Cuts_df['periods'] = periods # add a variable of number of forecasting periods 
New_Cuts_df['periods_for_acc_cal'] = 183 # add a variable of number of out of sample forecasting for accuracy 
New_Cuts_df['cutoff_month'] = cutoff_month # add a variable of the cutoff day
New_Cuts_df = New_Cuts_df.reset_index(drop=True)
New_Cuts_df

In [ ]:
new_cuts_df_FilePath = base_path + 'New_Cuts_df.csv'
New_Cuts_df.to_csv(new_cuts_df_FilePath, index=False)

# Accuracy with new hyper parameters

In [ ]:
calculator = AccuracyCalculator()
new_accuracy_df = calculator.CalculateAccuracy(New_Cuts_df, final_df, query_item_list)
#calculator.SaveFile()
calculator.SaveFile(output_bucket, folder_name, 'New Accuracy.csv')

In [ ]:
new_accuracy_df['Percentage Error'] = np.abs(new_accuracy_df['Percentage Error'])
new_accuracy = new_accuracy_df.groupby(['account_type', 'funnel']).mean(numeric_only=1) # group by split 
new_accuracy['Accuracy'] = np.where(new_accuracy['MAPE'] > 100, 'MAPE above 100%', 100 - new_accuracy['MAPE']) # create accuracy from MAPE 
new_accuracy['Forecast Period Accuracy'] = np.where(new_accuracy['Forecast Period MAE'] > 100, 'Forecast Period MAE above 100%', 100 - new_accuracy['Forecast Period MAE']) # create period accuracy from period MAPE
new_accuracy.drop(columns=['MAPE','Forecast Period MAE'], inplace=True) # drop error columns 
new_accuracy


## Prophet parameter search for Bing Ads Generic

In [ ]:
param_grid = {
    'changepoint_prior_scale': [ 0.1, 0.05, 0.001],   
    'seasonality_prior_scale': [10.0, 0.1, 0.01, 1.0 ], 
    'holidays_prior_scale': [10.0, 1.0,  0.1, 0.01  ], 
    'seasonality_mode': ['additive','multiplicative'], 
    'input_mode': ['additive','multiplicative'], 
    'weekday_or_weekend':['na', 'additive', 'multiplicative'],  
    'include_holiday': ['na', 'default', 'customized'],
    'covid_year_exclude':['na'],
    'covid_year_dummy':['na'],#, 'additive', 'multiplicative'],
    'anomaly_2021':['na'],#,, 'additive', 'multiplicative'
    'generic_cost_split':['na'],#, 'additive', 'multiplicative'
    'competitor_cost_split':['na'], #, 'additive', 'multiplicative'
    'new_confirmed': ['na'],#, 'additive', 'multiplicative'
    'January_Feb_2023':['na', 'additive', 'multiplicative']#,
            }


In [ ]:
# set the prophet parameter search with the hyper parameter grid created above and power flag as True (since the basic set up isnt returning good results)
search_engine = ProphetParameterSearch(param_grid = param_grid, power_flag = False, base_path=base_path) 
#search_engine.add_power_params() # consider using it when the basic setting won't get you good results
grid_brand = search_engine.search(New_Cuts_df.loc[(New_Cuts_df['account_type']=='Bing Ads')&(New_Cuts_df['funnel']=='Generic')], df) # performs the grid search over the hyper parameter grid. exports a best result .csv table with best hyper parameters by split and a record df .csv table with full hyper parameters results



## Prophet parameter search for Google AdWords Competitor

In [ ]:
param_grid = {
    'changepoint_prior_scale': [ 0.1, 0.05, 0.001],   
    'seasonality_prior_scale': [10.0, 0.1, 0.01, 1.0 ], 
    'holidays_prior_scale': [10.0, 1.0,  0.1, 0.01  ], 
    'seasonality_mode': ['additive','multiplicative'], 
    'input_mode': ['additive','multiplicative'], 
    'weekday_or_weekend':['na', 'additive', 'multiplicative'],  
    'include_holiday': ['na', 'default', 'customized'],
    'covid_year_exclude':['na'],
    'covid_year_dummy':['na'],#, 'additive', 'multiplicative'],
    'anomaly_2021':['na'],#, 'additive', 'multiplicative'
    'generic_cost_split':['na'],#, 'additive', 'multiplicative'
    'competitor_cost_split':['na'], #, 'additive', 'multiplicative'
    'new_confirmed': ['na'],#, 'additive', 'multiplicative'
    'January_Feb_2023':['na']#, 'additive', 'multiplicative'
            }


In [ ]:

# set the prophet parameter search with the hyper parameter grid created above and power flag as True (since the basic set up isnt returning good results)
search_engine = ProphetParameterSearch(param_grid = param_grid, power_flag = False, base_path=base_path) 
#search_engine.add_power_params() # consider using it when the basic setting won't get you good results
grid_brand = search_engine.search(New_Cuts_df.loc[(New_Cuts_df['account_type']=='Google AdWords')&(New_Cuts_df['funnel']=='Competitor')], df) # performs the grid search over the hyper parameter grid. exports a best result .csv table with best hyper parameters by split and a record df .csv table with full hyper parameters results



In [ ]:
# set the new cuts df 
Google_Competitor_Cuts = pd.read_csv(base_path + 'record_df 2024-04-30 12:41:45 best result.csv', index_col = 0) # read the best results df from prophet hyper parameter search
Google_Competitor_Cuts = Google_Competitor_Cuts.drop_duplicates() # drop duplicate rows, best result sometimes creates duplicate rows 
Google_Competitor_Cuts = Google_Competitor_Cuts.groupby(['market','account_type', 'funnel']).apply(lambda x: x[x['MAPE'] == x['MAPE'].min()], include_groups=False) # group by breakdowns and select only the minimum MAPE values
Google_Competitor_Cuts.reset_index(level=['market', 'account_type', 'funnel'], inplace=True)
Google_Competitor_Cuts['na_count'] = Google_Competitor_Cuts.map(lambda x: x == 'na').sum(axis=1) # count number of 'na' hyper parameters 
Google_Competitor_Cuts = Google_Competitor_Cuts.groupby(['market','account_type', 'funnel']).apply(lambda x: x[x['na_count'] == x['na_count'].max()], include_groups=False) # select the set of hyper parameters that has more 'na' for parsimony porpuses
Google_Competitor_Cuts.reset_index(level=['market', 'account_type', 'funnel'], inplace=True)
Google_Competitor_Cuts.drop(columns=['na_count','MAPE', 'period_AE'], inplace=True) # drop columns that are not needed in cuts_df
Google_Competitor_Cuts['periods'] = periods # add a variable of number of forecasting periods 
Google_Competitor_Cuts['periods_for_acc_cal'] = 183 # add a variable of number of out of sample forecasting for accuracy 
Google_Competitor_Cuts['cutoff_month'] = cutoff_month # add a variable of the cutoff day
Google_Competitor_Cuts = Google_Competitor_Cuts.reset_index(drop=True)
Google_Competitor_Cuts.drop(columns=['January_Feb_2023'], inplace=True)
# replace the new Bing Ads Generic hyper parameters
New_Cuts_df.loc[New_Cuts_df.loc[(New_Cuts_df['account_type']=='Google AdWords')&(New_Cuts_df['funnel']=='Competitor')].index] = Google_Competitor_Cuts.values[0]
New_Cuts_df


In [ ]:
param_grid = {
    'changepoint_prior_scale': [ 0.1, 0.05, 0.001],   
    'seasonality_prior_scale': [10.0, 0.1, 0.01, 1.0 ], 
    'holidays_prior_scale': [10.0, 1.0,  0.1, 0.01  ], 
    'seasonality_mode': ['additive','multiplicative'], 
    'input_mode': ['additive','multiplicative'], 
    'weekday_or_weekend':['na', 'additive', 'multiplicative'],  
    'include_holiday': ['na', 'default', 'customized'],
    'covid_year_exclude':['na'],
    'covid_year_dummy':['na'],#, 'additive', 'multiplicative'],
    'anomaly_2021':['na'],#, 'additive', 'multiplicative'
    'generic_cost_split':['na'],#, 'additive', 'multiplicative'
    'competitor_cost_split':['na'], #, 'additive', 'multiplicative'
    'new_confirmed': ['na'],#, 'additive', 'multiplicative'
    'January_Feb_2023':['na']#,, 'additive', 'multiplicative'
            }


In [ ]:

# set the prophet parameter search with the hyper parameter grid created above and power flag as True (since the basic set up isnt returning good results)
search_engine = ProphetParameterSearch(param_grid = param_grid, power_flag = False, base_path=base_path) 
#search_engine.add_power_params() # consider using it when the basic setting won't get you good results
grid_brand = search_engine.search(New_Cuts_df.loc[(New_Cuts_df['account_type']=='Bing Ads')&(New_Cuts_df['funnel']=='Competitor')], df) # performs the grid search over the hyper parameter grid. exports a best result .csv table with best hyper parameters by split and a record df .csv table with full hyper parameters results



In [ ]:
# set the new cuts df 
BingAds_Competitor_Cuts = pd.read_csv(base_path + 'record_df 2024-04-30 15:27:52 best result.csv', index_col = 0) # read the best results df from prophet hyper parameter search
BingAds_Competitor_Cuts = BingAds_Competitor_Cuts.drop_duplicates() # drop duplicate rows, best result sometimes creates duplicate rows 
BingAds_Competitor_Cuts = BingAds_Competitor_Cuts.groupby(['market','account_type', 'funnel']).apply(lambda x: x[x['MAPE'] == x['MAPE'].min()], include_groups=False) # group by breakdowns and select only the minimum MAPE values
BingAds_Competitor_Cuts.reset_index(level=['market', 'account_type', 'funnel'], inplace=True)
BingAds_Competitor_Cuts['na_count'] = BingAds_Competitor_Cuts.map(lambda x: x == 'na').sum(axis=1) # count number of 'na' hyper parameters 
BingAds_Competitor_Cuts = BingAds_Competitor_Cuts.groupby(['market','account_type', 'funnel']).apply(lambda x: x[x['na_count'] == x['na_count'].max()], include_groups=False) # select the set of hyper parameters that has more 'na' for parsimony porpuses
BingAds_Competitor_Cuts.reset_index(level=['market', 'account_type', 'funnel'], inplace=True)
BingAds_Competitor_Cuts.drop(columns=['na_count','MAPE', 'period_AE'], inplace=True) # drop columns that are not needed in cuts_df
BingAds_Competitor_Cuts['periods'] = periods # add a variable of number of forecasting periods 
BingAds_Competitor_Cuts['periods_for_acc_cal'] = 183 # add a variable of number of out of sample forecasting for accuracy 
BingAds_Competitor_Cuts['cutoff_month'] = cutoff_month # add a variable of the cutoff day
BingAds_Competitor_Cuts = BingAds_Competitor_Cuts.reset_index(drop=True)
BingAds_Competitor_Cuts.drop(columns=['January_Feb_2023'], inplace=True)
BingAds_Competitor_Cuts
# replace the new Bing Ads Generic hyper parameters
New_Cuts_df.loc[New_Cuts_df.loc[(New_Cuts_df['account_type']=='Bing Ads')&(New_Cuts_df['funnel']=='Competitor')].index] = BingAds_Competitor_Cuts.values[0]
New_Cuts_df


## Trick
##### When accuracy is too lower for certain splits, consider the recent year trend change, instead of running training from the first day of the history, only use the recent years might be helpful. be cautious if using this methodology. Rows in df will need to be dropped before creating pickle files.
##### Below is an example for Google Competitor. Dropping the history data before Jan 1, 2022 will generate better accuracy.  

In [ ]:
param_grid = {
    'changepoint_prior_scale': [ 0.1, 0.05, 0.001],   
    'seasonality_prior_scale': [10.0, 0.1, 0.01, 1.0 ], 
    'holidays_prior_scale': [10.0, 1.0,  0.1, 0.01  ], 
    'seasonality_mode': ['additive','multiplicative'], 
    'input_mode': ['additive','multiplicative'], 
    'weekday_or_weekend':['na', 'additive', 'multiplicative'],  
    'include_holiday': ['na', 'default', 'customized'],
    'covid_year_exclude':['na'],
    'covid_year_dummy':['na'],#, 'additive', 'multiplicative'],
    'anomaly_2021':['na'],#,, 'additive', 'multiplicative'
    'generic_cost_split':['na'],#, 'additive', 'multiplicative'
    'competitor_cost_split':['na'], #, 'additive', 'multiplicative'
    'new_confirmed': ['na'],#, 'additive', 'multiplicative'
    'January_Feb_2023':['na', 'additive', 'multiplicative']#,
            }


In [ ]:
# use the recent years might be helpful, filter data before 2022
temp_df = df[df.ds < cutoff_month].copy()
temp_df.drop(temp_df[(temp_df.account_type == 'Bing Ads') & (temp_df.funnel == 'Generic') & (temp_df.ds <= '2022-01-01')].index, inplace=True)


In [ ]:
# set the prophet parameter search with the hyper parameter grid created above and power flag as false (not needed)
# instead of running training from the first day of the history, only use the recent years might be helpful
# be cautious if using this methodolofy. rows in df will need to be dropped before creating pickle files

search_engine = ProphetParameterSearch(param_grid = param_grid, power_flag = False, base_path=base_path) 
grid_brand = search_engine.search(New_Cuts_df.loc[(New_Cuts_df['account_type']=='Bing Ads')&(New_Cuts_df['funnel']=='Generic')], temp_df) # performs the grid search over the hyper parameter grid. exports a best result .csv table with best hyper parameters by split and a record df .csv table with full hyper parameters results


Since the trick hyper parameter search generates hyper parameters with lower MAPE, replace previous hyper params in cuts df 

In [ ]:
# set the new cuts df 
BingAds_Generic_Cuts = pd.read_csv(base_path + 'record_df 2024-04-30 08:54:53 best result.csv', index_col = 0) # read the best results df from prophet hyper parameter search
BingAds_Generic_Cuts = BingAds_Generic_Cuts.drop_duplicates() # drop duplicate rows, best result sometimes creates duplicate rows 
BingAds_Generic_Cuts = BingAds_Generic_Cuts.groupby(['market','account_type', 'funnel']).apply(lambda x: x[x['MAPE'] == x['MAPE'].min()], include_groups=False) # group by breakdowns and select only the minimum MAPE values
BingAds_Generic_Cuts.reset_index(level=['market', 'account_type', 'funnel'], inplace=True)
BingAds_Generic_Cuts['na_count'] = BingAds_Generic_Cuts.map(lambda x: x == 'na').sum(axis=1) # count number of 'na' hyper parameters 
BingAds_Generic_Cuts = BingAds_Generic_Cuts.groupby(['market','account_type', 'funnel']).apply(lambda x: x[x['na_count'] == x['na_count'].max()], include_groups=False) # select the set of hyper parameters that has more 'na' for parsimony porpuses
BingAds_Generic_Cuts.reset_index(level=['market', 'account_type', 'funnel'], inplace=True)
BingAds_Generic_Cuts.drop(columns=['na_count','MAPE', 'period_AE'], inplace=True) # drop columns that are not needed in cuts_df
BingAds_Generic_Cuts['periods'] = periods # add a variable of number of forecasting periods 
BingAds_Generic_Cuts['periods_for_acc_cal'] = 183 # add a variable of number of out of sample forecasting for accuracy 
BingAds_Generic_Cuts['cutoff_month'] = cutoff_month # add a variable of the cutoff day
BingAds_Generic_Cuts = BingAds_Generic_Cuts.reset_index(drop=True)
# replace the new Bing Ads Generic hyper parameters
New_Cuts_df.loc[New_Cuts_df.loc[(New_Cuts_df['account_type']=='Bing Ads')&(New_Cuts_df['funnel']=='Generic')].index] = BingAds_Generic_Cuts.values[0]
New_Cuts_df

Export the new cuts df after the hyper parameter tuning

# Update new_cuts_df with new hyper parameters

In [ ]:
new_cuts_df_FilePath = base_path + 'New_Cuts_df.csv'
New_Cuts_df.to_csv(new_cuts_df_FilePath)

Since the trick hyper parameter search generates hyper parameters with lower MAPE, drop in df the data previous to 2022 for Bing Ads Generic

In [ ]:
# use the recent years might be helpful, filter data before 2022
df = df[df.ds < cutoff_month].copy()
df.drop(df[(temp_df.account_type == 'Bing Ads') & (df.funnel == 'Generic') & (df.ds <= '2022-01-01')].index, inplace=True)


# Model Generator

Generate pickle files where are stored the prophet model trained with the selected hyper parameters, the training data set and the complete data set

In [ ]:
# %load_ext autoreload
# %autoreload 
model_generator = GenerateModelFile()
model_generator.Generate(New_Cuts_df, df , query_item_list, output_bucket, folder_name, base_path=base_path)

# Exam

Exam the performance of 'US Bing Ads Generic' to understand the prediction trend since it is gettiong bad results

In [ ]:
# Import needed packages

import pickle
from repo.Accuracy import MakeFuture
from prophet.plot import plot_plotly, plot_components_plotly

In [ ]:
# Read the split pickle file

# set the pickle file name 
file_name = "US Bing Ads Generic.pkl"
pickle_file_path = base_path + file_name

# open the pickle file 
pickle_in = open(pickle_file_path,'rb') 

# unpickled model is the model to use, training_dataset is used to get historical spend
unpickled_model, training_dataset, model_df = pickle.load(pickle_in)


In [ ]:
# Set a copy of the analytical file and some splits variables

analytical_file_copy = analytical_file_df.copy() # set the analytical file copy
market = 'US' # set the first split breakdown variable value
account_type = 'Bing Ads' # set the second split breakdown variable value
funnel = 'Generic' # set the third split breakdown variable value


In [ ]:
# Filter the analytical file for just the selected split

spend_df_for_forecast = analytical_file_copy[(analytical_file_copy['market'] == market) & (analytical_file_copy['account_type'] == account_type) & \
                                              (analytical_file_copy['funnel'] == funnel)].copy()

spend_df_for_forecast['ds'] = pd.to_datetime(spend_df_for_forecast.ds)

for i in New_Cuts_df.loc[(New_Cuts_df['market']=='US') & (New_Cuts_df['account_type']=='Bing Ads') & (New_Cuts_df['funnel']=='Generic')].itertuples(): # get the itertuple for 
     pass 


In [ ]:
# Create the future data frame and predict using the prophet model

# create the future data frame
future = MakeFuture(model = unpickled_model, model_df = training_dataset, values = i, periods = 183, future_input_df = spend_df_for_forecast)
forecast = unpickled_model.predict(future) # predict the model with the future data frame


In [ ]:
# Plot components of the prophet forecast and plot the forecast

unpickled_model.plot_components(forecast) # plot components
plot_plotly(unpickled_model, forecast) # plot the forecast model

# Constraint Setting

In [ ]:
bounds = New_Cuts_df.copy()
bounds['MinWeight'] = 0.8
bounds['MaxWeight'] = 1.2
bounds

# %load_ext autoreload
# %autoreload 2
constraint_file_generator = ConstraintFileGenerator(config_dict, analytical_file_df)
constraint_file = constraint_file_generator.Generate(bounds)
constraint_file_generator.SaveFile(output_bucket, folder_name, constraint_file_name)

### When constraints are customized

In [ ]:
constraint_file.loc[constraint_file['Month'].isin(['Nov']), 'MinWeight'] = 0.7
constraint_file.loc[constraint_file['Month'].isin(['Nov']), 'MaxWeight'] = 0.8

constraint_file_generator.SaveFile(output_bucket, folder_name, constraint_file_name, df=constraint_file)

In [ ]:
stop

---

---

# Accuracy Calculator

In [76]:
# %load_ext autoreload
# %autoreload 2
calculator = AccuracyCalculator()
accuracy_df = calculator.CalculateAccuracy(cuts_df, final_df, query_item_list)
# calculator.SaveFile(bucket_name, folder_name)

17:51:15 - cmdstanpy - INFO - Chain [1] start processing
17:51:16 - cmdstanpy - INFO - Chain [1] done processing


It's underpredicted.
Monthly Error: 47.62 Period MAE: 44.7


/home/jupyter/SearchForecasting/repo/Accuracy.py:267: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.monthly_acc_df_temp = pd.concat([self.monthly_acc_df_temp, monthly_acc_df], ignore_index=True)
17:51:27 - cmdstanpy - INFO - Chain [1] start processing
17:51:28 - cmdstanpy - INFO - Chain [1] done processing


It's overpredicted.
Monthly Error: 90.21 Period MAE: 25.319999999999997


In [77]:
100-accuracy_df.groupby(['account_type', 'funnel']).mean(numeric_only=1)

Actual     Forecast  Percentage Error   MAPE  \
account_type funnel                                                          
-            Brand     -17446.000000 -9602.898571         55.624286  52.38   
             Non_Brand  -1072.142857 -1368.960000        177.167143   9.79   

                        Forecast Period MAE  
account_type funnel                          
-            Brand                    55.30  
             Non_Brand                74.68

In [ ]:
# PlotAccuracyCharts(accuracy_df, cuts_df, query_item_list)

# Model Generator

In [78]:
# %load_ext autoreload
# %autoreload 
model_generator = GenerateModelFile()
model_generator.Generate(cuts_df, final_df , query_item_list, output_bucket, folder_name, base_path=base_path)

17:53:54 - cmdstanpy - INFO - Chain [1] start processing
17:53:56 - cmdstanpy - INFO - Chain [1] done processing
17:53:58 - cmdstanpy - INFO - Chain [1] start processing
17:54:00 - cmdstanpy - INFO - Chain [1] done processing


# Constraint Setting

In [ ]:
%load_ext autoreload
%autoreload 
bounds = cuts_df.copy()
bounds['MinWeight'] = 0.8
bounds['MaxWeight'] = 1.2
bounds

# %load_ext autoreload
# %autoreload 2
constraint_file_generator = ConstraintFileGenerator(config_dict, analytical_file_df)
constraint_file = constraint_file_generator.Generate(bounds)

# Depends on buesiness requirements
constraint_file.loc[constraint_file.Month == 'Nov', 'MinWeight'] = 1
constraint_file.loc[constraint_file.Month == 'Nov', 'MaxWeight'] = 1.2

constraint_file.loc[constraint_file.Month == 'Oct', 'MinWeight'] = 0.6
constraint_file.loc[constraint_file.Month == 'Oct', 'MaxWeight'] = 0.8


constraint_file_generator.SaveFile(output_bucket, folder_name, constraint_file_name, df=constraint_file)

# Regional pre forecast total spend

In [ ]:
final_df[(final_df.ds>='2022-12-31')&(final_df.ds<='2023-02-01')].groupby(by='region').sum(numeric_only=1)

# Exam

In [ ]:
%load_ext autoreload
%autoreload 2

for i in cuts_df.itertuples():
    break

import pickle
from repo.Accuracy import MakeFuture
file_name = "Canada - Brand.pkl"
pickle_file_path = base_path + file_name
pickle_in = open(pickle_file_path,'rb') 
# unpickled model is the model to use, training_dataset is used to get historical spend
unpickled_model, training_dataset, model_df = pickle.load(pickle_in)

analytical_file_copy = analytical_file_df.copy()
market = 'Canada'
account_type = '-'
funnel = 'Brand'

spend_df_for_forecast = analytical_file_copy[(analytical_file_copy['market'] == market) & (analytical_file_copy['account_type'] == account_type) & \
                                             (analytical_file_copy['funnel'] == funnel)].copy()
spend_df_for_forecast['ds'] = pd.to_datetime(spend_df_for_forecast.ds)

future = MakeFuture(unpickled_model, training_dataset, i, 183, spend_df_for_forecast)
print(future.ds.max())
forecast = unpickled_model.predict(future)
unpickled_model.plot_components(forecast)

# from prophet.plot import plot_plotly, plot_components_plotly

# plot_plotly(unpickled_model, forecast)

In [ ]:
i